In [48]:
def loaddata():
    post = [['my','dog','has','flea','problems','help','please'],
           ['maybe','not','take','him','to','dog','park','stupid'],
           ['my','dalmation','is','so','cute','I','love','him'],
           ['stop','posting','stupid','worthless','garbage'],
           ['mr','licks','ate','my','steak','how','to','stop','him'],
           ['quit','buying','worthless','dog','food','stupid']]
    classvec = [0,1,0,1,0,1]
    return post,classvec
def vocablist(dataset):
    vocabset = set([])
    for document in dataset:
        vocabset = vocabset| set(document)
    return list(vocabset)
def vec(vocablist,inputset):
    returnvec = [0]*len(vocablist)
    for word in inputset:
        if word in vocablist:
            returnvec[vocablist.index(word)] = 1
        else: print('the word:%s is not in vocablist!'%word)
    return returnvec


In [2]:
post,classvec = loaddata()

In [6]:
vocablist = vocablist(post)


In [8]:
vec(vocablist,post[0])

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1]

In [29]:
# 朴素贝叶斯分类器训练函数，计算两个类别中每个单词的比例
import numpy as np
def train(matrix,category):
    num = len(matrix)#文档个数
    numword = len(matrix[0])#词汇个数
    p = sum(category)/float(num)#class为1的概率
    p0 = np.ones(numword)
    p1 = np.ones(numword)
    p0nom=2.0
    p1nom=2.0
    for i in range(num):
        if category[i]==1:#文档类别为1
            p1 += matrix[i]
            p1nom += sum(matrix[i])
        else:
            p0 += matrix[i]
            p0nom += sum(matrix[i])
    p1vect = np.log(p1/p1nom)#class为1计算每个单词的比例
    p0vect = np.log(p0/p0nom)#class为0计算每个单词的比例
    return p0vect,p1vect,p

In [10]:
trainmat=[]
for pos in post:
    trainmat.append(vec(vocablist,pos))

In [19]:
p0,p1,p = train(trainmat,classvec)


In [25]:
p


0.5

In [30]:
def classify(vec2,p0,p1,p):#vec2是输入的单词向量
    p1 = np.sum(vec2*p1) + np.log(p)
    p0 = np.sum(vec2*p0) + np.log(1-p)
    if p1 > p0: return 1
    else: return 0
def test(vocablist,p0,p1,p):
    testentry = ['love','my','dalmation']
    thisdoc = np.array(vec(vocablist,testentry))
    print('testentry1 classfied as:',classify(thisdoc,p0,p1,p))
    testentry = ['stupid','garbage']
    thisdoc = np.array(vec(vocablist,testentry))
    print('testentry2 classified as:',classify(thisdoc,p0,p1,p))
    

In [23]:
test(vocablist,p0,p1,p)

testentry1 classfied as: 0
testentry2 classified as: 1


In [31]:
#修改词集模型为词袋模型
def bagword(vocablist,inputset):
    returnvec = [0]*len(vocablist)
    for word in inputset:
        if word in vocablist:
            returnvec[vocablist.index(word)] += 1
    return returnvec

In [32]:
#文件解析及完整的垃圾邮件测试函数
import re 
def  textparse(bigstring):
    listoftoken = re.split('\W+',bigstring)
    return [tok.lower() for tok in listoftoken if len(tok)>2]


In [33]:
def spamtest():
    doclist=[]
    classlist=[]
    fulltext=[]
    for i in range(1,26):
        with open('/Users/enjlife/machine-learning/machinelearninginaction/ch04/email/spam/%d.txt'% i,
                 encoding='ISO-8859-1') as fr:
            wordlist = textparse(fr.read())
        doclist.append(wordlist)
        fulltext.extend(wordlist)
        classlist.append(1)
        with open('/Users/enjlife/machine-learning/machinelearninginaction/ch04/email/ham/%d.txt'% i,
                 encoding='ISO-8859-1') as fr:
            wordlist = textparse(fr.read())
        doclist.append(wordlist)
        fulltext.extend(wordlist)
        classlist.append(0)
    vocab = vocablist(doclist)
    trainset = list(range(50))
    testset=[]
    for i in range(10):
        idx = int(np.random.uniform(0,len(trainset)))
        testset.append(trainset[idx])
        del(trainset[idx])
    trainmat=[]
    trainclass=[]
    for idx in trainset:
        trainmat.append(vec(vocab,doclist[idx]))
        trainclass.append(classlist[idx])
    p0,p1,p = train(np.array(trainmat),np.array(trainclass))
    count = 0
    for idx in testset:
        wordve = np.array(vec(vocab,doclist[idx]))
#         print(p0.shape)
#         print(wordve.shape)
        if classify(wordve,p0,p1,p) !=classlist[idx]:
            print(doclist[idx])
            count +=1
    print('the error rate:%f'%(float(count)/len(testset)))
        
        

In [34]:
spamtest()

['home', 'based', 'business', 'opportunity', 'knocking', 'your', 'door', 'don', 'rude', 'and', 'let', 'this', 'chance', 'you', 'can', 'earn', 'great', 'income', 'and', 'find', 'your', 'financial', 'life', 'transformed', 'learn', 'more', 'here', 'your', 'success', 'work', 'from', 'home', 'finder', 'experts']
the error rate:0.100000


In [35]:
import feedparser
ny = feedparser.parse('https://www.craigslist.org/about/best/all/index.rss')

In [39]:
ny['entries'][0]['summary']


'Free snowman used only one season...bring your own bucket...<br>\n<br>'

In [71]:
import operator
def calc(vocab,fulltext):
    fd = {}
    for v in vocab:
        fd[v] = fulltext.count(v)
    sortedfreq = sorted(list(fd.items()),key=operator.itemgetter(1),reverse=True)
    return sortedfreq[:30]
#def localwords(feed1,feed0):#这里我们把一个分两份吧
def localwords(feed):#这里我们把一个分两份吧
    doclist=[]
    classlist=[]
    fulltext=[]
    #minlen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(12):
        
        wordlist = textparse(feed['entries'][i]['summary'])
        doclist.append(wordlist)
        fulltext.extend(wordlist)
        classlist.append(0)
        wordlist = textparse(feed['entries'][i+12]['summary'])
        doclist.append(wordlist)
        fulltext.extend(wordlist)
        classlist.append(1)
    vocab = vocablist(doclist)
    top30 = calc(vocab,fulltext)
    for pair in top30:
        if pair[0] in vocab: vocab.remove(pair[0])#去掉频率最高的30个词
    trainset = list(range(2*12))
    testset = []
    for i in range(4):
        idx = int(np.random.uniform(0,len(trainset)))
        testset.append(trainset[idx])
        del(trainset[idx])
    trainmat = []
    trainclass = []
    for idx in trainset:
        trainmat.append(bagword(vocab,doclist[idx]))
        trainclass.append(classlist[idx])
    p0,p1,p = train(np.array(trainmat),np.array(trainclass))
    count = 0
    for idx in testset:
        wordvec = bagword(vocab,doclist[idx])
        if classify(np.array(wordvec),p0,p1,p) != classlist[idx]:
            count +=1
            print(doclist[idx])
    print('the error rate:%f'%(float(count)/len(testset)))
    return vocab,p0,p1

In [72]:
localwords(ny)

['hello', 'bicycle', 'weirdo', 'converted', 'car', 'powered', 'bicycle', 'you', 'can', 'see', 'the', 'pictures', 'the', 'engine', 'has', 'been', 'replaced', 'bike', 'the', 'original', 'transmission', 'now', 'has', 'cassette', 'and', 'still', 'shifts', 'and', 'are', 'gone', 'the', 'rest', 'the', 'car', 'has', 'been', 'stripped', 'down', 'leaving', 'only', 'the', 'rolling', 'bits', 'exterior', 'driver', 'seat', 'brakes', 'and', 'steering', 'think', 'currently', 'weighs', 'about', '900lbs', 'the', 'flats', 'easier', 'pedal', 'than', 'expected', 'biked', 'tiny', 'hill', 'the', 'cargo', 'bike', 'requires', 'two', 'operate', 'one', 'pedal', 'and', 'one', 'steer', 'have', 'plans', 'make', 'single', 'operator', 'attaching', 'controls', 'the', 'handlebars', 'but', 'the', 'project', 'stalled', 'when', 'moved', 'the', 'car', 'los', 'altos', 'hills', 'figured', 'waste', 'leave', 'this', 'sitting', 'someone', 'weird', 'enough', 'want', 'take', 'the', 'playa', 'functional', 'but', 'certainly', 'not'

(['owned',
  'switch',
  'end',
  'bidding',
  'manufacturer',
  'axe',
  'tug',
  'trident',
  'requesting',
  'lucky',
  'exaggerate',
  'friend',
  'three',
  'mouth',
  '1998',
  'gets',
  'lips',
  'cord',
  'favorite',
  'infant',
  'excellent',
  'flops',
  'previous',
  'crespi',
  'stomp',
  'adding',
  'standing',
  'long',
  'downtown',
  'drives',
  'eyes',
  'see',
  'maserati',
  'joe',
  'drop',
  'hair',
  'outlive',
  'local',
  'yesterday',
  'creatures',
  'given',
  'yelling',
  'hillary',
  'inch',
  'bridges',
  'enough',
  'lay',
  'economic',
  'bored',
  'washer',
  'rebuild',
  'ago',
  'pictures',
  'paying',
  'done',
  'bel',
  'include',
  'society',
  'disassembled',
  'lost',
  'own',
  'attaching',
  'act',
  'powers',
  'weird',
  'socks',
  'tons',
  'forgive',
  'impressed',
  'caveat',
  'halfway',
  'each',
  'lend',
  'title',
  'writing',
  'detroit',
  'removal',
  'burning',
  'proudly',
  'santa',
  'cold',
  'require',
  'deductible',
  'appr

In [77]:
def getword(ny):
    vocab,p0,p1 = localwords(ny)
    topny = []
    topsf = []
    for i in range(len(p0)):
        if p0[i] > -6.0: topny.append((vocab[i],p0[i]))
        #print(p1[i])
        if p1[i] > -6.0: topny.append((vocab[i],p1[i]))
    sortsf = sorted(topsf,key=lambda pair:pair[1],reverse=True)
    print('---------------------sf-----------------------')
    for item in sortsf:
        print(item[0])
    sortny = sorted(topny,key=lambda pair:pair[1],reverse=True)
    print('---------------------ny-----------------------')
    for item in sortny:
        print(item[0])
    

In [78]:
getword(ny)

['free', 'snowman', 'used', 'only', 'one', 'season', 'bring', 'your', 'own', 'bucket']
['apple', 'macintosh', '512', 'plus', 'stone', 'casting', 'old', 'macintosh', 'didn', 'turn', 'stone', 'when', 'saw', 'wearing', 'apple', 'watch', 'this', 'high', 'quality', 'casting', 'early', '1980', 'classic', 'can', 'the', 'material', 'but', 'looks', 'lot', 'like', 'concrete', 'just', 'haven', 'seen', 'concrete', 'castings', 'this', 'quality', 'showing', 'this', 'much', 'detail', 'truly', 'one', 'kind', 'piece', 'for', 'the', 'hard', 'core', 'macintosh', 'fan', 'evangelist', 'don', 'let', 'this', 'one', 'slip', 'past', 'you', 'you', 'may', 'not', 'see', 'another', 'one', 'sorry', 'keyboard', 'mouse', 'boot', 'disks', 'software', 'cord', 'little', 'dirty', 'excellent', 'condition', 'only', 'chip', 'could', 'find', 'pictured', 'below', 'very', 'heavy', 'and', 'solid', 'the', 'rock', 'serious', 'mac', 'inquiries', 'only']
the error rate:0.500000
---------------------sf-----------------------
-------